# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your https://jupyterhub.ischool.syr.edu/ workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

# Part 1: MapReduce with Spark

In [1]:
# Run this code to create the Spark session
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('map-reduce').getOrCreate()
sc = spark.sparkContext

In [2]:
# this RDD will be used throughout this part of the homework
gpa_rdd = sc.parallelize([
 ['2015', 'Fall', 'IST101', 1, 'A', 4],
 ['2015', 'Fall', 'IST195', 3, 'A', 4],
 ['2015', 'Fall', 'IST233', 3, 'B+', 3.3],
 ['2015', 'Fall', 'SOC101', 3, 'A-', 3.7],
 ['2015', 'Fall', 'MAT221', 3, 'C', 2],
 ['2016', 'Fall', 'IST346', 3, 'A', 4],
 ['2016', 'Fall', 'CHE111', 4, 'A-', 3.7],
 ['2016', 'Fall', 'PSY120', 3, 'B+', 3.3],
 ['2016', 'Fall', 'IST256', 3, 'A', 4],
 ['2016', 'Fall', 'ENG121', 3, 'B+', 3.3],
 ['2016', 'Spring', 'GEO110', 3, 'B+', 3.3],
 ['2016', 'Spring', 'MAT222', 3, 'A', 4],
 ['2016', 'Spring', 'SOC121', 3, 'C+', 2.3],
 ['2016', 'Spring', 'BIO240', 3, 'B-', 2.7]])

## Question 1.1 Cumulative GPA with MapReduce (15 pts)

The cumulative GPA reported in any transcript is the average of the numerical score of a course weighted by the credits of such course. Construct a MapReduce job that takes the `gpa_rdd` RDD and returns the cumulative GPA *per season*. 

Each record in `gpa_rdd` contains:
- The year
- The season
- The course code
- The number of credits
- The letter grade
- The number grade

**Hint:** In class, we discussed the MapReduce job for computing avereage. In this case, the key-value pairs will be similar but instead of counting the number of elements considered in the average so far, we need to count the credits. Clearly, the key needs to be the semester.

In [3]:
def map_weighted_gpa(record): # Defined map_weighted_gpa() map function
    year, season, course_code, number_of_credits, letter_grade, number_grade = record # Defined what each record contains
    c = number_of_credits # Assigned number_of_credits as c
    g = number_grade # Assigned number_grade as g
    return [season, [g,c]] # Returned season as key, with g and c as a tuple of values

def reduce_weighted_gpa(value1, value2): # Defined reduce_weighted_gpa() reduce function
    g1, c1 = value1 # Defined what value 1 contains
    g2, c2 = value2 # Defined what value 2 contains
    c3 = c1 + c2 # Total number of credits is equal to the number of credits from both value1 and value 2
    g3 = (g1*c1 + g2*c2)/c3 # Average equals the grade multipled by the number of credits from value 1 added to the grade multipled by the number of credits from value 2 divided by the total number of credits
    return [ 
        g3
        ,
        c3
    ] # Returned weighted gpa and number of credits

The map job should produce as key the season and value a tuple or list with the grade and credit.

For example, the first element of the map of `gpa_rdd` should be

```python
gpa_rdd.\
    map(map_weighted_gpa).\
    first()
```

```python
['Fall', [4, 1]]
```

In [4]:
gpa_rdd.\
    map(map_weighted_gpa).\
    first()
['Fall', [4, 1]]

['Fall', [4, 1]]

In [5]:
# 15 pts
##### first result
assert gpa_rdd.\
    map(map_weighted_gpa).\
    first() == ['Fall', [4, 1]]
# the key should be a string
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: type(x[0])).distinct().count() == 1
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: type(x[0])).distinct().first() == str
# all values should be of length 2
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: len(x[1])).distinct().count() == 1
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: len(x[1])).distinct().first() == 2

In [6]:
# there should be two results in the map reduce because there are two semesters
gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).take(2)

[('Fall', [3.503448275862069, 29]), ('Spring', [3.0749999999999997, 12])]

In [7]:
# 5 pts
assert (gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).collect() == \
    [('Spring', [3.0749999999999997, 12]), 
     ('Fall', [3.503448275862069, 29])]) or \
    (gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).collect() == \
    [('Fall', [3.503448275862069, 29]),
     ('Spring', [3.0749999999999997, 12])])

## Question 1.2 Time series

Historical changes in temperature throughout history is a higly noisy process. Still, we can use long-term trends to understand these fluctiations and perhaps relate them to external factors such as changes in the oceans, magnetic fields of the earth, or human activity (CO2).

Luckily, we have data to explore how temperatures have changes over the years. In the dataset below, we have obtained a set of key-value pairs where the key is the year $y$ and the value is a temperature deviation $t_y$ from a long-term historical record (e.g., inferred from the artic's permafrost).

In [8]:
global_avg_temp = sc.parallelize([[1880, -0.06], [1881, -0.09], [1882, 0.08], [1883, -0.28], 
                                  [1884, -0.24], 
                   [1885, -0.45], [1886, -0.18], [1887, -0.53], [1888, -0.33], [1889, -0.03],
                   [1890, -0.34], [1891, -0.4], [1892, -0.25], [1893, -0.74], [1894, -0.42],
                   [1895, -0.44], [1896, -0.15], [1897, -0.13], [1898, -0.01], [1899, -0.14],
                   [1900, -0.35], [1901, -0.13], [1902, -0.1], [1903, -0.24], [1904, -0.62],
                   [1905, -0.35], [1906, -0.14], [1907, -0.37], [1908, -0.38], [1909, -0.58],
                    [1910, -0.36], [1911, -0.55], [1912, -0.28], [1913, -0.35], [1914, 0.11],
                   [1915, -0.15], [1916, -0.13], [1917, -0.53], [1918, -0.32], [1919, -0.23],
                    [1920, -0.2], [1921, -0.07], [1922, -0.3], [1923, -0.24], [1924, -0.22],
                    [1925, -0.4], [1926, 0.13], [1927, -0.23], [1928, -0.08], [1929, -0.49],
                    [1930, -0.34], [1931, -0.03], [1932, 0.15], [1933, -0.29], [1934, -0.3],
                    [1935, -0.28], [1936, -0.27], [1937, -0.18], [1938, 0.02], [1939, -0.11],
                    [1940, 0.01], [1941, 0.27], [1942, 0.29], [1943, -0.03], [1944, 0.39],
                    [1945, 0.16], [1946, 0.24], [1947, -0.16], [1948, 0.06], [1949, 0.12],
                   [1950, -0.29], [1951, -0.28], [1952, 0.13], [1953, 0.09], [1954, -0.2],
                    [1955, 0.14], [1956, -0.14], [1957, -0.09], [1958, 0.32], [1959, 0.16],
                   [1960, 0.04], [1961, 0.13], [1962, 0.12], [1963, 0.06], [1964, 0.01],
                    [1965, -0.07], [1966, -0.05], [1967, -0.09], [1968, -0.2], [1969, -0.09],
                    [1970, 0.14], [1971, 0.01], [1972, -0.24], [1973, 0.28], [1974, -0.19],
                    [1975, 0.11], [1976, -0.02], [1977, 0.13], [1978, 0.16], [1979, 0.15],
                    [1980, 0.33], [1981, 0.51], [1982, 0.14], [1983, 0.53], [1984, 0.3],
                    [1985, 0.22], [1986, 0.31], [1987, 0.32], [1988, 0.56], [1989, 0.17],
                    [1990, 0.36], [1991, 0.43], [1992, 0.46], [1993, 0.36], [1994, 0.27],
                    [1995, 0.56], [1996, 0.25], [1997, 0.34], [1998, 0.6], [1999, 0.51],
                   [2000, 0.34], [2001, 0.47], [2002, 0.71], [2003, 0.72], [2004, 0.61],
                    [2005, 0.65], [2006, 0.51], [2007, 0.92], [2008, 0.27], [2009, 0.6],
                    [2010, 0.73], [2011, 0.47], [2012, 0.44], [2013, 0.62],
                   [2014, 0.7], [2015, 0.83], [2016, 1.12], [2017, 0.98], [2018, 0.76],
                   [2019, 0.94], [2020, 1.15], [2021, 0.8]])

## Question 1.2.1: Year with lowest temperature deviation (10 pts)

In this question, you will compute the year with the minimum temperature deviation using MapReduce. The key-value pair structure of the map reduce should have a single key and you should use the string `"min"` for it. The first element of the value at the end of the computation should have the year and the second element should be the lowest temperature deviation.

In [9]:
# define the map operation
def argmin_map(kv): # Defined argin_map() map function
    year, deviation = kv # Defined what each key value pair contains
    y = year # Defined year as y
    d = deviation # Defined deviation as d
    return ["min", [y,d]] # Returned "min" as key, with y and d as a tuple of values

def argmin_reduce(v1, v2): # Defined argmin_reduce() reduce function
    y1, d1 = v1 # Defined what v1 contains
    y2, d2 = v2 # Defined what v2 contains
    d3 = min(d1,d2) # Set d3 as the lower of the two deviations using the min() function
    if d3 == d1: # If d3 equals d1:
        y3 = y1 # Assigned y1 as the minimum year
    else: # Else:
        y3 = y2 # Assigned y2 as the minimum year
    return [  
        y3
        ,
        d3
    ] # Returned year with minimum deviation

In [10]:
global_avg_temp.map(argmin_map).reduceByKey(argmin_reduce).take(1)

[('min', [1893, -0.74])]

In [11]:
# 10 pts
# there should be 1 result only
assert global_avg_temp.map(argmin_map).reduceByKey(argmin_reduce).count() == 1
# the resulting value should have two elements (year, lowest temperature)
assert len(global_avg_temp.map(argmin_map).reduceByKey(argmin_reduce).values().first()) == 2
# lowest temperature happened in year 1893
assert global_avg_temp.map(argmin_map).reduceByKey(argmin_reduce).values().first()[0] == 1893
# lowest temperature was -0.74
assert global_avg_temp.map(argmin_map).reduceByKey(argmin_reduce).values().first()[1] == -0.74

## Question 1.2.2: Running average of temperature deviation (25 pts)

Sometimes, time series are too noisy and require smoothing. We can do this smoothing by computing a running average of values. In this question, you will compute a simple moving average where the resulting RDD will have as key the year $y$ and as the first element of the value $m_y$, the average temperature deviation of the last $n$ years in the data (including the current year):

$$m_y = \frac{\sum_{i=y-n+1}^{y} t_i}{n}$$

For example, for the input data and for $n=2$

```
(1880, -0.06)
(1881, -0.09)
(1882, 0.08)
(1883, -0.28)
```

The first element for key 1883 will be

$$
\begin{array}
mm_{1883} &=\frac{\sum_{i=1883-2+1}^{1883} t_i}{2}\\
& =\frac{\sum_{i=1882}^{1883} t_i}{2}\\
& =\frac{0.08 -0.28}{2}\\
& =0.1
\end{array}$$

Below, define a function that computes the moving average for $n=5$. Notice that you will be able to compute the moving average **only** starting in year 1884 and ending in year 2021. Because each input element should likely generate several key-value pairs, use `flatMap` instead of `map` as your map step.

In [12]:
# define the map operation
def ma_map(kv): # Defined ma_map() map function
    n = 5 # Set number of years as equal to 5
    x = 1 # Set default count as 1
    deviation = [] # Created empty deviation list
    for year in range(n): # For year in range(n) 1,2,3,4,5...
        deviation.append([kv[0] + year, [kv[1], x]]) # Append the first value of the key,value pair adding each value of n as the key and the second value of the key,value pair along with x (1) as a tuple of values
    return(deviation) # Returned year as key, with deviation and count as a tuple of values over the course of 5 years

def ma_reduce(v1, v2): # Defined ma_reduce() reduce function
    g1, c1 = v1 # Defined what v1 contains
    g2, c2 = v2 # Defined what v2 contains
    c3 = c1 + c2 # Used same average function as used in question 1.1
    g3 = (g1*c1 + g2*c2)/c3 
    return [ 
        g3
        ,
        c3
    ] # Returned moving average and year

In [13]:
# the first result of the moving average should be average of the first five values of the temperature deviations
sum(global_avg_temp.values().take(5))/5

-0.11800000000000002

In [14]:
# confirm that you get the same results
sc.parallelize(global_avg_temp.take(5)).\
    flatMap(ma_map).\
    reduceByKey(ma_reduce).\
    filter(lambda kv: kv[0] == 1884).\
    map(lambda kv: (kv[0], kv[1][0])).\
    collect()

[(1884, -0.118)]

If you were to run the following code, it should love like this:

```python
global_avg_temp.\
    flatMap(ma_map).\
    reduceByKey(ma_reduce).\
    filter(lambda kv: kv[0] >= 1884 and kv[0] <= 2021).\
    map(lambda kv: [kv[0], kv[1][0]]).\
    sortByKey().\
    take(10)
```

```
[(1884, -0.11800000000000002),
 (1885, -0.196),
 (1886, -0.21400000000000002),
 (1887, -0.33599999999999997),
 (1888, -0.346),
 (1889, -0.30400000000000005),
 (1890, -0.28200000000000003),
 (1891, -0.32600000000000007),
 (1892, -0.27),
 (1893, -0.352)]
 ```

In [15]:
# try it here
global_avg_temp.\
    flatMap(ma_map).\
    reduceByKey(ma_reduce).\
    filter(lambda kv: kv[0] >= 1884 and kv[0] <=2021).\
    map(lambda kv: [kv[0], kv[1][0]]).\
    sortByKey().\
    take(10)

[(1884, -0.11800000000000002),
 (1885, -0.196),
 (1886, -0.21400000000000002),
 (1887, -0.33599999999999997),
 (1888, -0.346),
 (1889, -0.30400000000000005),
 (1890, -0.28200000000000003),
 (1891, -0.32600000000000007),
 (1892, -0.27),
 (1893, -0.352)]

In [16]:
# 25 pts
np.testing.assert_almost_equal(global_avg_temp.\
    flatMap(ma_map).\
    reduceByKey(ma_reduce).\
    filter(lambda kv: kv[0] >= 1884 and kv[0] <= 2021).\
    map(lambda kv: [kv[0], kv[1][0]]).\
    sortByKey().\
    values().sum(), 7.807999999999996)